Задание 1.4

Вам необходимо оптимизировать расход бумаги в принтере, который зависит от часов работы двух ключевых сотрудников исходя из функции f.

Где `x` — количество часов работы 1-ого сотрудника, `y` — 2-ого сотрудника.

Однако вы должны оплатить этим сотрудникам ровно 20 часов работы в сумме.

Какое наименьшее количество листов бумаги вы сможете потратить в таком случае?

Помните, что количество часов работы должно быть целым, поэтому, прежде чем вычислять итоговый результат, округлите часы работы до целого.

In [168]:
from sympy import * 

In [169]:
x, y, l = symbols('x y lambda')
f = x**2 + 2 * y**2
f

x**2 + 2*y**2

Функция ограничений

In [170]:
g = x + y - 20
g

x + y - 20

Использую метод Лагранжа

In [171]:
L = f + l * g
L

lambda*(x + y - 20) + x**2 + 2*y**2

Определение частных производных по x, y, l

In [172]:
L_diff_x = L.diff(x)
L_diff_x

lambda + 2*x

In [173]:
L_diff_y = L.diff(y)
L_diff_y

lambda + 4*y

In [174]:
L_diff_l = L.diff(l)
L_diff_l

x + y - 20

Составляю систем уравнений приравнивая частные производные к нулю

In [175]:
eq0 = Eq(L_diff_x, 0)
eq1 = Eq(L_diff_y, 0)
eq2 = Eq(L_diff_l, 0)

sol = solve([eq0, eq1, eq2], [x, y, l])

Получаю значения x и y для заданного ограничения

In [176]:
sol

{lambda: -80/3, x: 40/3, y: 20/3}

Округляю часы работы до целых

In [177]:
round(40/3)

13

In [178]:
round(20/3)

7

Вычисляю основную зависимость исходя из найденных часов работы

In [179]:
f.subs({x: 13, y: 7})

267

### Задание 2.7

Давайте потренируемся применять стохастический градиентный спуск для решения задачи линейной регрессии. 

Мы уже рассмотрели его реализацию «с нуля», однако для решения практических задач можно использовать готовые библиотеки.

In [180]:
import numpy as np
import pandas as pd
import seaborn as sns

# Загрузите стандартный датасет об алмазах из библиотеки Seaborn
df = sns.load_dataset('diamonds')

In [181]:
# Удалите часть признаков
df.drop(['depth', 'table', 'x', 'y', 'z'], axis=1, inplace=True)

In [184]:
# Закодируйте категориальные признаки
df = pd.get_dummies(df, drop_first=True)

In [185]:
# Логарифмируйте признаки
df['carat'] = np.log(1 + df['carat'])
df['price'] = np.log(1 + df['price'])

In [186]:
# Определите целевую переменную и предикторы
TARGET = 'price'

X, y = df.drop(columns=TARGET), df[TARGET]

Разделите выборку на обучающую и тестовую (объём тестовой возьмите равным 0.33), значение random_state должно быть равно 42.

Теперь реализуйте алгоритм линейной регрессии со стохастическим градиентным спуском (класс SGDRegressor). 

Отберите с помощью GridSearchCV оптимальные параметры по следующей сетке:

In [187]:
param_grid = {'loss': ['squared_error', 'epsilon_insensitive'],
              'penalty': ['elasticnet'],
              'alpha': np.logspace(-3, 3, 10),
              'l1_ratio': np.linspace(0, 1, 10),
              'learning_rate': ['constant'],
              'eta0': np.logspace(-4, -1, 4)}

In [188]:
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error

from sklearn import set_config
set_config(transform_output='pandas')

In [189]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [196]:
model = SGDRegressor(
        random_state=42, # генератор случайных чисел
        # max_iter=100000 # количество итераций на сходимость
    )

grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    cv=5,
    n_jobs=-1,
    scoring='neg_mean_squared_error'
)

%time grid_search.fit(X_train, y_train) 
y_train_pred = grid_search.predict(X_train)
y_test_pred = grid_search.predict(X_test)

print(f'Наилучшие значения гиперпараметров: {grid_search.best_params_}')
print(f'Наилучшее значение метрики на валидационной выборке: {grid_search.best_score_}')

CPU times: total: 19.1 s
Wall time: 59.4 s
Наилучшие значения гиперпараметров: {'alpha': 0.001, 'eta0': 0.001, 'l1_ratio': 0.0, 'learning_rate': 'constant', 'loss': 'epsilon_insensitive', 'penalty': 'elasticnet'}
Наилучшее значение метрики на валидационной выборке: -0.04481974544218573


In [197]:
print(f'MSE on train: {mean_squared_error(y_train, y_train_pred):.3f}')
print(f'MSE on test: {mean_squared_error(y_test, y_test_pred):.3f}')

MSE on train: 0.045
MSE on test: 0.044
